In [8]:
import numpy as np

In [9]:
# Setup: project path and OMDb API key
import os
import sys
from pathlib import Path
import numpy as np

# Ensure project root is on sys.path so `import app` works when running from notebooks/
PROJECT_ROOT = Path.cwd().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Configure OMDb API key for experiments.
# Prefer setting OMDB_API_KEY in your environment or `.env` file.
# For quick experiments, you can set it here (replace the placeholder):
OMDB_API_KEY = os.environ.get("OMDB_API_KEY") or "YOUR_OMDB_API_KEY_HERE"
os.environ["OMDB_API_KEY"] = OMDB_API_KEY
print("OMDB_API_KEY set:", "present" if bool(os.environ.get("OMDB_API_KEY")) and os.environ.get("OMDB_API_KEY") != "YOUR_OMDB_API_KEY_HERE" else "missing")

OMDB_API_KEY set: present


In [10]:
# Imports, async runner, and loading the recommender
import asyncio
from typing import Any

from app.services.recommender import recommend_movies


def run(coro):
    """Run an async coroutine from a notebook cell, returning its result."""
    try:
        loop = asyncio.get_event_loop()
    except RuntimeError:
        loop = None
    if loop and loop.is_running():
        # If already in an event loop (e.g., IPython), create a new task
        return asyncio.ensure_future(coro)
    return asyncio.run(coro)


def show_results(items: list[dict[str, Any]]):
    if not items:
        print("No results.")
        return
    for i, m in enumerate(items, 1):
        print(f"{i}. {m.get('title')}  |  Rating: {m.get('vote_average')}  |  Released: {m.get('release_date')}")
        if m.get("overview"):
            print(f"   {m['overview'][:200]}{'...' if len(m['overview'])>200 else ''}")
        if m.get("poster_url"):
            print(f"   Poster: {m['poster_url']}")
        print()


In [11]:
# Example 1: Keyword strategy (default)
MOOD = "gory fun"
LIMIT = 6

movies = run(recommend_movies(mood=MOOD, limit=LIMIT, strategy="keyword"))

# If we're in a running loop (Jupyter), `run` may return a Task. Await it to get results.
try:
    from asyncio import Future
    if hasattr(movies, "__await__") or isinstance(movies, Future):
        movies = await movies
except Exception:
    pass

show_results(movies)


1. Little Horror Movie  |  Rating: 4.1  |  Released: 28 Jun 2019
   Three You tubers struggling to get more views on their adventure channel, travel to the mythical city of Casablanca, unaware that it hides a terrifying secret.
   Poster: https://m.media-amazon.com/images/M/MV5BYmM4NzAwN2EtZTRhNy00ZGYyLTljNzEtZGY0OWM0N2FhYWEyXkEyXkFqcGc@._V1_SX300.jpg

2. The Amityville Horror  |  Rating: 6.2  |  Released: 27 Jul 1979
   Newlyweds and their children battle a demonic presence in their home.
   Poster: https://m.media-amazon.com/images/M/MV5BNWRmOTdhMWEtYmExOC00Y2E5LTkwYjEtYTAwY2Q0MTU0MTBlXkEyXkFqcGc@._V1_SX300.jpg

3. The Rocky Horror Picture Show  |  Rating: 7.4  |  Released: 29 Sep 1975
   A newly-engaged couple have a breakdown in an isolated area and must seek shelter at the bizarre residence of Dr. Frank-n-Furter.
   Poster: https://m.media-amazon.com/images/M/MV5BOTg2YzY5ZGYtNDk1My00N2Q2LWFhN2YtZWU5YTkzODIyZGRmXkEyXkFqcGc@._V1_SX300.jpg

4. Horror in the High Desert  |  Rating: 5.

In [12]:
# Example 2: Embedding/TF-IDF strategy
MOOD = "claustrophobic psychological"
LIMIT = 6

movies = run(recommend_movies(mood=MOOD, limit=LIMIT, strategy="embedding"))

try:
    from asyncio import Future
    if hasattr(movies, "__await__") or isinstance(movies, Future):
        movies = await movies
except Exception:
    pass

show_results(movies)


1. A Classic Horror Story  |  Rating: 5.7  |  Released: 14 Jul 2021
   In this gruesome suspense film, strangers traveling in southern Italy become stranded in the woods, where they must fight desperately to get out alive.
   Poster: https://m.media-amazon.com/images/M/MV5BYTg5ZDgxNDAtMzFkNS00M2UwLWI4N2EtOTczYjM0MjYwZDRjXkEyXkFqcGc@._V1_SX300.jpg

2. The Rocky Horror Picture Show: Let's Do the Time Warp Again  |  Rating: 4.2  |  Released: 20 Oct 2016
   A straitlaced, square couple, seeking shelter from a storm, find themselves in the castle of a transgender alien mad scientist intent on creating a buff bodybuilder.
   Poster: https://m.media-amazon.com/images/M/MV5BN2FjZDExZDgtZTE2NS00MTU0LWEzNTAtMGZiZDZjYmE0NjYyXkEyXkFqcGc@._V1_SX300.jpg

3. #Horror  |  Rating: 3.1  |  Released: 20 Nov 2015
   Six preadolescent girls face a night of terror when the compulsive addiction of an online social media game turns a moment of cyber bullying into a night of insanity.
   Poster: https://m.media

In [7]:
import gc
del OMDB_API_KEY
gc.collect()


25